In [1]:
import numpy as np 
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
import cv2
from sklearn.metrics import confusion_matrix, classification_report

Cargar las imagenes del dataset

In [ ]:
train_dir = './chest_xray/train/'
test_dir = './chest_xray/test/'
val_dir = './chest_xray/val/'

img_width = 160
img_height = 160

train_data = tf.keras.utils.image_dataset_from_directory(
    train_dir,
    seed = 123,
    image_size = (img_width, img_height),
    batch_size=None
)

test_data = tf.keras.utils.image_dataset_from_directory(
    test_dir,
    seed = 123,
    image_size = (img_width, img_height),
    batch_size=None 
)
val_data = tf.keras.utils.image_dataset_from_directory(
    val_dir,
    seed = 123,
    image_size = (img_width, img_height),
    batch_size=None
)

Guardar las imagenes de entrenamiento y validación 

In [ ]:
x_train = []
y_train = []
x_val = []
y_val = []
x_test = []
y_test = []

for feature, label in train_data:
    imagen = cv2.resize(feature.numpy(), (img_width, img_height))
    imagen = cv2.cvtColor(imagen, cv2.COLOR_BGR2GRAY)
    imagen = imagen.reshape(img_width, img_height, 1)
    x_train.append(imagen)
    y_train.append(label.numpy())

for feature, label in test_data:
    imagen = cv2.resize(feature.numpy(), (img_width, img_height))
    imagen = cv2.cvtColor(imagen, cv2.COLOR_BGR2GRAY)
    imagen = imagen.reshape(img_width, img_height, 1)
    x_test.append(imagen)
    y_test.append(label.numpy())
    
for feature, label in val_data:
    imagen = cv2.resize(feature.numpy(), (img_width, img_height))
    imagen = cv2.cvtColor(imagen, cv2.COLOR_BGR2GRAY)
    imagen = imagen.reshape(img_width, img_height, 1)
    x_val.append(imagen)
    y_val.append(label.numpy())

X_fit = np.array(x_train)
y_fit = np.array(y_train)
X_test = np.array(x_test)
y_test = np.array(y_test)
X_val = np.array(x_val)
y_val = np.array(y_val)
X = np.concatenate((X_fit, X_test, X_val))
y = np.concatenate((y_fit, y_test, y_val))



Split para entrenamiento y validacion -
Entrenamiento: 80%
Validacion: 20%

In [4]:
X_entrenamiento = X[:4685]
X_validacion = X[1171:]

y_entrenamiento = y[:4685]
y_validacion = y[1171:]

Aumento de datos y arquitectura de la red

In [ ]:
img_size = 160 #Tamaño de la imagen

#Capa de redimensión
resize_and_rescale = tf.keras.Sequential([
    tf.keras.layers.Resizing(img_size, img_size),
    tf.keras.layers.Rescaling(1./255)
])
#Capa de aumento de datos
data_augmentation = tf.keras.Sequential([
  tf.keras.layers.RandomFlip("horizontal_and_vertical"),
  tf.keras.layers.RandomRotation(factor=0.2),

])
#Capas convolucionales y densas
modelo = tf.keras.Sequential([
    resize_and_rescale,
    data_augmentation, 
    
    tf.keras.layers.Conv2D(16, (3,3), activation='relu', padding='same'),
    tf.keras.layers.MaxPooling2D((2,2), padding='same'), 

    tf.keras.layers.Conv2D(32, (3,3), activation='relu', padding='same'),
    tf.keras.layers.MaxPooling2D((2,2), padding='same'), 
        
    tf.keras.layers.Conv2D(16, (3,3), activation='relu', padding='same'),
    tf.keras.layers.MaxPooling2D((2,2), padding='same'), 
        
    tf.keras.layers.Flatten(),
    
    tf.keras.layers.Dense(256, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(2, activation='softmax')
])
modelo.compile(
    optimizer='adam',
    loss=tf.losses.SparseCategoricalCrossentropy(from_logits=False),
    metrics=['accuracy']
)


Entremiento

In [ ]:
reduce_lr = keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.2,
                              patience=5, min_lr=0.001)

history = modelo.fit(
    X_entrenamiento,
    y_entrenamiento,
    validation_data = (X_validacion, y_validacion),
    epochs = 20,
    callbacks=[reduce_lr]
    
)

Graficas del entrenamiento

In [ ]:
plt.figure()
plt.plot(history.history['val_accuracy'], label='val_acc')
plt.plot(history.history['accuracy'], label='train_acc')
plt.title('Precisión de clasificación')
plt.ylabel('Precisión')
plt.xlabel('Numero de entrenamientos')
plt.legend(loc='lower left')

plt.figure()
plt.plot(history.history['loss'], label='train_loss')
plt.plot(history.history['val_loss'], label = 'val_loss')
plt.title('Pérdida durante el entrenamiento')
plt.xlabel('Número de entrenamientos')
plt.ylabel('Pérdida')
plt.legend(loc='lower left')
modelo.summary()
tf.keras.utils.plot_model(modelo)

Guardar el modelo

In [24]:
modelo.save('my_model.keras')

Metricas

In [64]:
y_prediction = modelo.predict(X_test)
y_prediction = np.argmax(y_prediction, axis=1)
conf_m = confusion_matrix(y_test, y_prediction , normalize='pred')
print(conf_m)
clas_r = classification_report(y_test, y_prediction)
print(clas_r)

20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
[[0.97087379 0.2571977 ]
 [0.02912621 0.7428023 ]]
              precision    recall  f1-score   support

           0       0.97      0.43      0.59       234
           1       0.74      0.99      0.85       390

    accuracy                           0.78       624
   macro avg       0.86      0.71      0.72       624
weighted avg       0.83      0.78      0.75       624

